Programme de lancement de différent modèle

Librairies

In [5]:
import math
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from lightgbm import LGBMRegressor
from statsmodels.tsa.arima_model import ARIMA

from numpy import sort
from numpy import loadtxt

from xgboost import to_graphviz
from xgboost import plot_tree
from xgboost import plot_importance
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from xgboost import Booster

from sklearn import tree
from sklearn import linear_model
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LassoLarsIC
from sklearn.linear_model import SGDRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SelectFromModel

'ok'

'ok'

In [6]:
#Création de liste des modèles et de ses noms associés

Model=[] 
NameModel=[]
Model.append(MLPClassifier())
NameModel.append('MLPClassifier')
Model.append(LGBMRegressor())
NameModel.append('LGBMRegressor')
Model.append(XGBRegressor())
NameModel.append('XGBRegressor')
Model.append(RandomForestRegressor(random_state=2))
NameModel.append('RandomForestRegressor')
Model.append(MLPRegressor())
NameModel.append('MLPRegressor')
#Model.append(linear_model.LinearRegression())
#NameModel.append('linear_model.LinearRegression')
Model.append(tree.DecisionTreeRegressor())
NameModel.append('tree.DecisionTreeRegressor')
Model.append(tree.DecisionTreeClassifier())
NameModel.append('tree.DecisionTreeClassifier')
Model.append(tree.ExtraTreeRegressor())
NameModel.append('tree.ExtraTreeRegressor')
Model.append(GradientBoostingRegressor())
NameModel.append('GradientBoostingRegressor')
#Model.append(LassoLarsIC())
#NameModel.append('LassoLarsIC')
#Model.append(SGDRegressor())
#NameModel.append('SGDRegressor')
#Model.append(LinearRegression())
#NameModel.append('LogisticRegression')
#Model.append(LinearRegression())
#NameModel.append('LogisticRegression')

print('Nombre de modèle: {:d}'.format(len(Model)))

Nombre de modèle: 9


Cross Validation du modèle

In [7]:
#Chargement des données

dataframe = pd.read_csv('Data\BDDLAST.csv', engine='python', )
dataset = dataframe.values

#Création des variables

X = dataframe.drop('quantiteVendue', axis=1)
y = dataframe['quantiteVendue']

#Numéro du modèle à utiliser

NumModel=2

#On définit la MAPE qui sert quantifier l'erreur: pourcentage moyen de différence

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)   
    a = np.array(y_true)
    b = np.array(y_pred)
    mean = (a + b)/2.0
    return 100*np.mean(np.abs((y_true - y_pred) / np.maximum(mean, 1)))

#On définit la RMSE qui sert quantifier l'erreur: valeur moyenne de différence

def root_mean_square_error(y_pred, y_true):
    return math.sqrt(mean_squared_error(y_pred, y_true))

#Création des différents splits

# X_groups = pd.qcut(X.index, 100, labels=False)
pred = pd.Series([np.nan] * len(X), index=X.index)

n_predictions = 10
periode_prediction = 30

tss = TimeSeriesSplit(n_splits=n_predictions)
m_err=0
for train_index, test_index in tss.split(X): #, groups=X_groups):
    test_index = test_index[0:periode_prediction]
    
    X_train = X.loc[train_index]
    y_train = y.loc[train_index]
    X_test = X.loc[test_index]
    y_test = y.loc[test_index]
    
    #Entrainement du modèle choisi
    
    model=Model[NumModel]
    model.fit(X_train, y_train)

    pred.loc[test_index] = model.predict(X_test)

#Calucul de l'erreur
    
predicted_index = ~pred.isnull()
relative_error = mean_absolute_error(y[predicted_index], pred[predicted_index]) / np.mean(y)
mape=mean_absolute_percentage_error(y[predicted_index], pred[predicted_index])
print('Relative error for model {}: {:.2f}% '.format(model.__class__.__name__, relative_error*100))
print('Relative MAPE error for model: {:.2f}% '.format(mape))

Relative error for model XGBRegressor: 19.33% 
Relative MAPE error for model: 19.53% 


Split "taille" pour tester le modèle

In [9]:
#80 de valeur train et 20 de test

taille=len(X)*0.80

#Création de toutes les variables

X_train = X[dataframe.index < taille]
y_train = y[dataframe.index < taille]

X_test = X[dataframe.index >= taille]
y_test = y[dataframe.index >= taille]

#Entrainement du modèle

model = Model[NumModel]
model.fit(X_train, y_train)

#Récupération des données prédites

predictions = model.predict(X_test)

error = None
mape = mean_absolute_percentage_error(y_test, predictions)
rmse = root_mean_square_error(y_test, predictions)
print('Relative MAPE error for model: {:.2f}% '.format(mape))
print('Relative RMSE error for model: {:.2f} '.format(rmse))

Relative MAPE error for model: 17.06% 
Relative RMSE error for model: 929.20 
